In [1]:
#!pip install openpyxl 
from __future__ import print_function, unicode_literals, absolute_import, division
import sys
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from glob import glob
from tifffile import imread
from csbdeep.utils import Path, normalize
import cv2
import os

from stardist import random_label_cmap, _draw_polygons
from stardist.models import StarDist2D
from stardist import matching 
import stardist
import pandas as pd
np.random.seed(6)
lbl_cmap = random_label_cmap()

import openpyxl

Using TensorFlow backend.


In [2]:
#Choose model folder and test data folder
modelFolder='D:/Archivos/Trabajo/Nantes/Algorithms/KubernetesModels/iterModels_07_10_2020'
testFolder='D:/Archivos/Trabajo/Nantes/Algorithms/github-stardist/examples/2D/data/data/test'

In [32]:
#Preparing the test data
X0 = sorted(glob(testFolder+'/images/*.tif'))

X0 = list(map(imread,X0))

n_channel0 = 1 if X0[0].ndim == 2 else X0[0].shape[-1]
axis_norm0 = (0,1)   # normalize channels independently
# axis_norm = (0,1,2) # normalize channels jointly
if n_channel0 > 1:
    print("Normalizing image channels %s." % ('jointly' if axis_norm4 is None or 2 in axis_norm4 else 'independently'))

X0Mask = sorted(glob(testFolder+'/masks/*.tif'))
X0Mask = list(map(imread,X0Mask))

n_channel0 = 1 if X0[0].ndim == 2 else X0[0].shape[-1]
axis_norm0 = (0,1)   # normalize channels independently
# axis_norm = (0,1,2) # normalize channels jointly
if n_channel0 > 1:
    print("Normalizing image channels %s." % ('jointly' if axis_norm0 is None or 2 in axis_norm0 else 'independently'))

#Check the number of models
numModels=len(glob(modelFolder+'/stardist_*')) 

#Initialize a list for the results
results=[]
for t in range(numModels):
    
    modName=list(['stardist_'+str(t+1)])
    
    model = StarDist2D(None, name=modName[0], basedir=modelFolder)
    

    #Creation of predicted labels
    predLabels=list()
    predDetails=list()
    precisions=list()
    recalls=list()
    accuracies=list()
    f1s=list()
    mean_true_scores=list()

    for j in range(len(X0)):
        img = normalize(X0[j], 1,99.8, axis=axis_norm0)
        labels, details = model.predict_instances(img)
        predLabels.append(labels)
        predDetails.append(details)
    
        scores=matching.matching_dataset(X0Mask[j],predLabels[j])
    
        precisions.append(scores.precision)
        recalls.append(scores.recall)
        accuracies.append(scores.accuracy)
        f1s.append(scores.f1)
        mean_true_scores.append(scores.mean_true_score)


    precList=list([np.mean(precisions),np.std(precisions),precisions])
    #print('precision (mean=',np.mean(precisions),'+-',np.std(precisions),'):\n',precisions,'\n')
    recallList=list([np.mean(recalls),np.std(recalls),recalls])
    #print('recall (mean=',np.mean(recalls),'+-',np.std(recalls),'):\n',recalls,'\n')
    accList=list([np.mean(accuracies),np.std(accuracies),accuracies])
    #print('accuracy (mean=',np.mean(accuracies),'+-',np.std(accuracies),'):\n',accuracies,'\n\n')
    f1List=list([np.mean(f1s),np.std(f1s),f1s])
    #print('f1(mean=',np.mean(f1s),'+-',np.std(f1s),'):\n',f1s,'\n\n')
    mean_true_scoreList=list([np.mean(mean_true_scores),np.std(mean_true_scores),mean_true_scores])
    #print('mean_true_score(mean=',np.mean(mean_true_scores),'+-',np.std(mean_true_scores),'):\n',mean_true_scores,'\n\n')  
    
    #Read hyperparameter values
    file=open(modelFolder+'/params_'+str(t+1)+'.txt','r')
    params=list([file.read()])
    file.close()
    
    listResults=modName+params+precList+recallList+accList+f1List+mean_true_scoreList
    results.append(listResults)

columns=['Model name', 'Parameters','Precision (mean)', 'Precision (std)','Precisions','Recall (mean)','Recall (std)','Recalls',
         'Accuracy (mean)','Accuracy (std)', 'Accuracies','f1 (mean)', 'f1 (std)','f1s','Mean true score (mean)',
         'Mean true score (std)','Mean true scores']
resultsDF=pd.DataFrame(results,columns=columns)
resultsDF
resultsDF.to_excel(modelFolder+'/results.xlsx')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.446979, nms_thresh=0.5.



100%|██████████████████████████████████████████████████████████████████████████████| 512/512 [00:00<00:00, 1361.72it/s]


Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.531117, nms_thresh=0.3.


100%|██████████████████████████████████████████████████████████████████████████████| 512/512 [00:00<00:00, 1323.27it/s]


Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.403206, nms_thresh=0.3.


...

100%|██████████████████████████████████████████████████████████████████████████████| 512/512 [00:00<00:00, 1212.10it/s]


Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.545906, nms_thresh=0.3.


100%|███████████████████████████████████████████████████████████████████████████████| 512/512 [00:00<00:00, 790.65it/s]


Loading network weights from 'weights_best.h5'.
Couldn't load thresholds from 'thresholds.json', using default values. (Call 'optimize_thresholds' to change that.)
Using default values: prob_thresh=0.5, nms_thresh=0.4.


100%|███████████████████████████████████████████████████████████████████████████████| 512/512 [00:00<00:00, 783.98it/s]


In [3]:
#Analyze the best performing algorithms

resultsDF=pd.read_excel(modelFolder+'/results.xlsx')

resultsDF

,Unnamed: 0,Model name,Parameters,Precision (mean),Precision (std),Precisions,Recall (mean),Recall (std),Recalls,Accuracy (mean),Accuracy (std),Accuracies,f1 (mean),f1 (std),f1s,Mean true score (mean),Mean true score (std),Mean true scores
0,0,stardist_1,python stardist_train.py --path data/train --q...,0.867450,0.041548,"[0.9143695014662757, 0.8403193612774451, 0.791...",0.820260,0.061321,"[0.8743690409422322, 0.8752598752598753, 0.777...",0.730572,0.070096,"[0.8081907724209435, 0.750445632798574, 0.6450...",0.842417,0.046834,"[0.8939220183486238, 0.8574338085539714, 0.784...",0.682829,0.061769,"[0.7367121593581334, 0.7343935348311396, 0.641..."
1,1,stardist_2,python stardist_train.py --path data/train --q...,0.843548,0.041945,"[0.8777777777777778, 0.8520971302428256, 0.755...",0.780799,0.059931,"[0.8418395961862031, 0.8024948024948025, 0.706...",0.683152,0.063701,"[0.7535140562248996, 0.7043795620437956, 0.574...",0.810054,0.044931,"[0.8594331520183224, 0.8265524625267666, 0.730...",0.640988,0.057556,"[0.6901000557647939, 0.662308370667245, 0.5804..."
2,2,stardist_3,python stardist_train.py --path data/train --q...,0.847260,0.045758,"[0.8668711656441718, 0.8459821428571429, 0.735...",0.759362,0.072510,"[0.7924845765563657, 0.7879417879417879, 0.663...",0.670319,0.079243,"[0.7065, 0.6890909090909091, 0.535684987694831...",0.799988,0.055659,"[0.8280105479050689, 0.8159311087190527, 0.697...",0.616707,0.070824,"[0.6462835570916344, 0.6506182129715684, 0.541..."
3,3,stardist_4,python stardist_train.py --path data/train --q...,0.859123,0.042821,"[0.8946716232961586, 0.8419864559819413, 0.790...",0.769142,0.068020,"[0.8098710039259674, 0.7754677754677755, 0.708...",0.685078,0.075440,"[0.7393753200204813, 0.676950998185118, 0.5962...",0.810782,0.052078,"[0.8501619075654989, 0.8073593073593074, 0.747...",0.630002,0.065770,"[0.666951429144708, 0.6574412665089014, 0.5778..."
4,4,stardist_5,python stardist_train.py --path data/train --q...,0.824062,0.050087,"[0.860969387755102, 0.8302325581395349, 0.7375...",0.728803,0.083532,"[0.7571508693213684, 0.7422037422037422, 0.616...",0.634203,0.090042,"[0.6746626686656672, 0.644404332129964, 0.5058...",0.772562,0.065440,"[0.8057296329453895, 0.7837541163556532, 0.671...",0.582294,0.078066,"[0.6035006674115677, 0.6087504472614796, 0.492..."
5,5,stardist_6,python stardist_train.py --path data/train --q...,0.843773,0.038571,"[0.8777852802160703, 0.8696682464454977, 0.772...",0.702698,0.073586,"[0.7291082445316882, 0.762993762993763, 0.6056...",0.623249,0.072166,"[0.6619144602851323, 0.6847014925373134, 0.513...",0.765517,0.053711,"[0.7965686274509803, 0.8128460686600222, 0.678...",0.562177,0.067483,"[0.5833247751407082, 0.6236619446599567, 0.487..."
6,6,stardist_7,python stardist_train.py --path data/train --q...,0.859204,0.037989,"[0.9039145907473309, 0.812133072407045, 0.7962...",0.795800,0.071194,"[0.854739203589456, 0.8627858627858628, 0.6473...",0.705714,0.073453,"[0.7835475578406169, 0.7192374350086655, 0.555...",0.825261,0.051386,"[0.8786393773421736, 0.8366935483870968, 0.714...",0.659158,0.069148,"[0.7163339805639906, 0.7250882922620245, 0.537..."
7,7,stardist_8,python stardist_train.py --path data/train --q...,0.837649,0.038518,"[0.8851702250432776, 0.8539094650205762, 0.756...",0.815644,0.065089,"[0.860347728547392, 0.8627858627858628, 0.7276...",0.706035,0.068503,"[0.7739656912209889, 0.7518115942028986, 0.589...",0.825783,0.047558,"[0.8725824800910125, 0.8583247156153051, 0.741...",0.661838,0.059017,"[0.7064798499809932, 0.7146092710118043, 0.595..."
8,8,stardist_9,python stardist_train.py --path data/train --q...,0.860427,0.042506,"[0.8794326241134752, 0.9044117647058824, 0.776...",0.734312,0.064815,"[0.765002804262479, 0.7671517671517671, 0.6544...",0.658452,0.073498,"[0.6923857868020304, 0.7096153846153846, 0.550...",0.791737,0.052395,"[0.8182363527294542, 0.8301462317210349, 0.710...",0.598180,0.066845,"[0.6219268332217746, 0.6391103863567545, 0.519..."
9,9,stardist_10,python st

In [6]:
for i in range(len(resultsDF)):
    print(resultsDF.iloc[i]['Parameters'])

python stardist_train.py --path data/train --quick_demo False --model_dir iterModels --model_name stardist_1 --epochs 278 --grid 2 2 --train_learning_rate 0.000251 --n_rays 30 --unet_batch_norm True --unet_dropout 0.0 
python stardist_train.py --path data/train --quick_demo False --model_dir iterModels --model_name stardist_2 --epochs 219 --grid 2 2 --train_learning_rate 0.001102 --n_rays 29 --unet_batch_norm False --unet_dropout 0.30000000000000004 
python stardist_train.py --path data/train --quick_demo False --model_dir iterModels --model_name stardist_3 --epochs 246 --grid 4 4 --train_learning_rate 0.005062 --n_rays 30 --unet_batch_norm False --unet_dropout 0.30000000000000004 
python stardist_train.py --path data/train --quick_demo False --model_dir iterModels --model_name stardist_4 --epochs 264 --grid 4 4 --train_learning_rate 0.009512 --n_rays 22 --unet_batch_norm True --unet_dropout 0.2 
python stardist_train.py --path data/train --quick_demo False --model_dir iterModels --mod

In [7]:
#Printing the training parameters with the model with the highest precision
print('Model stardist_'+str(resultsDF['Precision (mean)'].argmax()+1)+' (maximum Precision)')
print('\n'+resultsDF.iloc[resultsDF['Precision (mean)'].argmax()]['Parameters'])

#Printing the training parameters with the model with the highest recall
print('\n\nModel stardist_'+str(resultsDF['Recall (mean)'].argmax()+1)+' (maximum Recall)')
print('\n'+resultsDF.iloc[resultsDF['Recall (mean)'].argmax()]['Parameters'])

#Printing the training parameters with the model with the highest accuracy
print('\n\nModel stardist_'+str(resultsDF['Accuracy (mean)'].argmax()+1)+' (maximum Accuracy)')
print('\n'+resultsDF.iloc[resultsDF['Accuracy (mean)'].argmax()]['Parameters'])

#Printing the training parameters with the model with the highest f1
print('\n\nModel stardist_'+str(resultsDF['f1 (mean)'].argmax()+1)+' (maximum f1)')
print('\n'+resultsDF.iloc[resultsDF['f1 (mean)'].argmax()]['Parameters'])

#Printing the training parameters with the model with the highest mean true score
print('\n\nModel stardist_'+str(resultsDF['Mean true score (mean)'].argmax()+1)+' (maximum Mean true score)')
print('\n'+resultsDF.iloc[resultsDF['Mean true score (mean)'].argmax()]['Parameters'])

Model stardist_16 (maximum Precision)

python stardist_train.py --path data/train --quick_demo False --model_dir iterModels --model_name stardist_16 --epochs 290 --grid 2 2 --train_learning_rate 0.005253 --n_rays 21 --unet_batch_norm False --unet_dropout 0.1 


Model stardist_1 (maximum Recall)

python stardist_train.py --path data/train --quick_demo False --model_dir iterModels --model_name stardist_1 --epochs 278 --grid 2 2 --train_learning_rate 0.000251 --n_rays 30 --unet_batch_norm True --unet_dropout 0.0 


Model stardist_1 (maximum Accuracy)

python stardist_train.py --path data/train --quick_demo False --model_dir iterModels --model_name stardist_1 --epochs 278 --grid 2 2 --train_learning_rate 0.000251 --n_rays 30 --unet_batch_norm True --unet_dropout 0.0 


Model stardist_1 (maximum f1)

python stardist_train.py --path data/train --quick_demo False --model_dir iterModels --model_name stardist_1 --epochs 278 --grid 2 2 --train_learning_rate 0.000251 --n_rays 30 --unet_batch_norm